# Group 17 DSCI Project (Section 007)
### Predicting diabetes based on demographic attributes, medical history, and clinical measurements

Darby Quinn #19752633 <br>
Manav Parikh #13928775 <br>
Nitya Goel #89433221 <br>
Reeva Bansal #68061514 <br>

#### Introduction

Diabetes is a common condition that affects people of all ages. In individuals with diabetes, the body either doesn't make enough insulin or can’t use insulin properly. This can cause serious health problems such as heart disease, vision loss, kidney disease and high blood pressure (BP). There are certain medical factors (e.g. BMI, age, insulin and glucose levels) that can help predict whether an individual is likely to have or develop diabetes. Using these factors and making predictions on whether someone has diabetes can help ensure that proper precautions are taken or treatments are administered to manage the negative short and long term effects of diabetes.

The question we are answering is: **Can we classify whether or not someone has diabetes based on their blood pressure, BMI and age?**

The dataset we are using to answer this question is a Diabetes dataset with nine different columns: number of pregnancies; blood glucose level; BP; skin thickness; blood insulin level; BMI; diabetes pedigree function; age; and outcome. This dataset is pre-divided into training and testing data, and has previously been useful in using classification to predict diabetes outcome, as well as studying risk factors and diabetes management. 

In [24]:
# loading libraries needed to perform analysis
library(tidyverse)
library(dplyr)
library(repr)
library(tidymodels)

#### Methods & Results

In [28]:
#reading in training data
url <- "https://raw.githubusercontent.com/nityag11/DSCI-100-group17-project/main/Training%20(1).csv"
download.file(url, "Training%20(1).csv")
training_data <- read.csv("Training%20(1).csv")
head(training_data)

# reading in testing data
url <- "https://raw.githubusercontent.com/nityag11/DSCI-100-group17-project/main/Testing.csv"
download.file(url, "Testing.csv")
testing_data <- read.csv("Testing.csv")
head(testing_data)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1
6,5,116,74,0,0,25.6,0.201,30,0


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,9,120,72,22,56,20.8,0.733,48,0
2,1,71,62,0,0,21.8,0.416,26,0
3,8,74,70,40,49,35.3,0.705,39,0
4,5,88,78,30,0,27.6,0.258,37,0
5,10,115,98,0,0,24.0,1.022,34,0
6,0,124,56,13,105,21.8,0.452,21,0
